In [ ]:
import cv2
import time
import csv
from ultralytics import YOLO
import numpy as np
import os
import psutil


VIDEO_PATH = "recortes\Frente_5m.mp4"
BW_OPTION = False
OUTPUT_CSV = 'Estadisticas\EstadisticasL2_5m_nano_Color.csv'
OUTPUT_TXT = 'Estadisticas\EstadisticasL2_5m_nano_Color.txt'


model = YOLO("yolov8n-pose.pt")  # Ligero y rápido


cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("❌ No se pudo abrir el video.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

min_size = min(width, height)
scale_w = min_size / width
scale_h = min_size / height

print(f"Video original: {width}x{height}, FPS: {fps}, Frames: {total_frames}")
print(f"Resolución ajustada a: {int(width*scale_w)}x{int(height*scale_h)}")


csv_headers = ["frame",
               "dist_euclid_shlders", "dist_manh_shlders",
               "dist_euclid_left_shoulder_hip", "dist_manh_left_shoulder_hip",
               "dist_euclid_right_shoulder_hip", "dist_manh_right_shoulder_hip",
               "dist_euclid_hips", "dist_manh_hips",
               "time_frame_sec", "cpu_percent", "memory_percent"]

csv_file = open(OUTPUT_CSV, mode='w', newline='')
writer = csv.writer(csv_file)
writer.writerow(csv_headers)


frame_num = 0
tiempo_total = 0
process = psutil.Process(os.getpid())

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Redimensionar
    frame_resized = cv2.resize(frame, (int(width*scale_w), int(height*scale_h)))

    # Convertir a B/N si se eligió
    if BW_OPTION:
        gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
        proc_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    else:
        proc_frame = frame_resized

    start_frame_time = time.time()  # Solo conteo inferencia + distancias

    # Detección de keypoints
    results = model(proc_frame, verbose=False)

    # Inicializar distancias
    dist_euclid_shlders = dist_manh_shlders = 0
    dist_euclid_lsh_hip = dist_manh_lsh_hip = 0
    dist_euclid_rsh_hip = dist_manh_rsh_hip = 0
    dist_euclid_hips = dist_manh_hips = 0

    for result in results:
        keypoints = result.keypoints.xy.cpu().numpy()
        if len(keypoints) > 0:
            person = keypoints[0]
            ls = person[5].astype(float)
            rs = person[6].astype(float)
            lh = person[11].astype(float)
            rh = person[12].astype(float)

            # Distancias Euclidiana
            dist_euclid_shlders = np.linalg.norm(ls - rs)
            dist_euclid_lsh_hip = np.linalg.norm(ls - lh)
            dist_euclid_rsh_hip = np.linalg.norm(rs - rh)
            dist_euclid_hips = np.linalg.norm(lh - rh)

            # Distancias Manhattan
            dist_manh_shlders = np.sum(np.abs(ls - rs))
            dist_manh_lsh_hip = np.sum(np.abs(ls - lh))
            dist_manh_rsh_hip = np.sum(np.abs(rs - rh))
            dist_manh_hips = np.sum(np.abs(lh - rh))

    tiempo_frame = time.time() - start_frame_time
    tiempo_total += tiempo_frame

    # Recursos medidos independientemente
    cpu_percent = process.cpu_percent(interval=0.0)/psutil.cpu_count()
    memory_percent = process.memory_percent()

    # Guardar en CSV
    writer.writerow([frame_num,
                     dist_euclid_shlders, dist_manh_shlders,
                     dist_euclid_lsh_hip, dist_manh_lsh_hip,
                     dist_euclid_rsh_hip, dist_manh_rsh_hip,
                     dist_euclid_hips, dist_manh_hips,
                     tiempo_frame,
                     cpu_percent, memory_percent])

    # Mostrar frame
    cv2.imshow("Video con Keypoints", frame_resized if not BW_OPTION else gray_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_num += 1

cap.release()
cv2.destroyAllWindows()
csv_file.close()

tiempo_video_real = total_frames / fps
factor = tiempo_total / tiempo_video_real

estadisticas = f"""
Frames procesados: {frame_num}/{total_frames}
Duración del video real: {tiempo_video_real:.2f} seg
Tiempo total de procesamiento (solo inferencia y cálculo distancias): {tiempo_total:.2f} seg
Factor de procesamiento vs tiempo real: {factor:.2f}x
CSV guardado en: {OUTPUT_CSV}
"""

print(estadisticas)

# Guardar estadísticas en TXT
with open(OUTPUT_TXT, "w") as f:
    f.write(estadisticas)

print(f"📄 Estadísticas finales guardadas en: {OUTPUT_TXT}")


<>:10: SyntaxWarning: invalid escape sequence '\L'
<>:12: SyntaxWarning: invalid escape sequence '\E'
<>:13: SyntaxWarning: invalid escape sequence '\E'
<>:10: SyntaxWarning: invalid escape sequence '\L'
<>:12: SyntaxWarning: invalid escape sequence '\E'
<>:13: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17212\240629094.py:10: SyntaxWarning: invalid escape sequence '\L'
  VIDEO_PATH = "recortes\Lado2_5m.mp4"
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17212\240629094.py:12: SyntaxWarning: invalid escape sequence '\E'
  OUTPUT_CSV = 'Estadisticas\EstadisticasL2_5m_nano_Color.csv'
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17212\240629094.py:13: SyntaxWarning: invalid escape sequence '\E'
  OUTPUT_TXT = 'Estadisticas\EstadisticasL2_5m_nano_Color.txt'


Video original: 848x478, FPS: 23.0, Frames: 14
Resolución ajustada a: 477x478

Frames procesados: 14/14
Duración del video real: 0.61 seg
Tiempo total de procesamiento (solo inferencia y cálculo distancias): 1.20 seg
Factor de procesamiento vs tiempo real: 1.98x
CSV guardado en: Estadisticas\EstadisticasL2_5m_nano_Color.csv

📄 Estadísticas finales guardadas en: Estadisticas\EstadisticasL2_5m_nano_Color.txt


In [10]:
import cv2
import time
import csv
from ultralytics import YOLO
import numpy as np
import os
import psutil

VIDEO_PATH = "recortes\Lado2_5m.mp4"
BW_OPTION = True
OUTPUT_CSV = 'Estadisticas\EstadisticasL2_5m_nano_ByN.csv'
OUTPUT_TXT = 'Estadisticas\EstadisticasL2_5m_nano_ByN.txt'

model = YOLO("yolov8n-pose.pt")  # Ligero y rápido


cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("❌ No se pudo abrir el video.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

min_size = min(width, height)
scale_w = min_size / width
scale_h = min_size / height

print(f"Video original: {width}x{height}, FPS: {fps}, Frames: {total_frames}")
print(f"Resolución ajustada a: {int(width*scale_w)}x{int(height*scale_h)}")


csv_headers = ["frame",
               "dist_euclid_shlders", "dist_manh_shlders",
               "dist_euclid_left_shoulder_hip", "dist_manh_left_shoulder_hip",
               "dist_euclid_right_shoulder_hip", "dist_manh_right_shoulder_hip",
               "dist_euclid_hips", "dist_manh_hips",
               "time_frame_sec", "cpu_percent", "memory_percent"]

csv_file = open(OUTPUT_CSV, mode='w', newline='')
writer = csv.writer(csv_file)
writer.writerow(csv_headers)


frame_num = 0
tiempo_total = 0
process = psutil.Process(os.getpid())

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Redimensionar
    frame_resized = cv2.resize(frame, (int(width*scale_w), int(height*scale_h)))

    # Convertir a B/N si se eligió
    if BW_OPTION:
        gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
        proc_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    else:
        proc_frame = frame_resized

    start_frame_time = time.time()  # Solo conteo inferencia + distancias

    # Detección de keypoints
    results = model(proc_frame, verbose=False)

    # Inicializar distancias
    dist_euclid_shlders = dist_manh_shlders = 0
    dist_euclid_lsh_hip = dist_manh_lsh_hip = 0
    dist_euclid_rsh_hip = dist_manh_rsh_hip = 0
    dist_euclid_hips = dist_manh_hips = 0

    for result in results:
        keypoints = result.keypoints.xy.cpu().numpy()
        if len(keypoints) > 0:
            person = keypoints[0]
            ls = person[5].astype(float)
            rs = person[6].astype(float)
            lh = person[11].astype(float)
            rh = person[12].astype(float)

            # Distancias Euclidiana
            dist_euclid_shlders = np.linalg.norm(ls - rs)
            dist_euclid_lsh_hip = np.linalg.norm(ls - lh)
            dist_euclid_rsh_hip = np.linalg.norm(rs - rh)
            dist_euclid_hips = np.linalg.norm(lh - rh)

            # Distancias Manhattan
            dist_manh_shlders = np.sum(np.abs(ls - rs))
            dist_manh_lsh_hip = np.sum(np.abs(ls - lh))
            dist_manh_rsh_hip = np.sum(np.abs(rs - rh))
            dist_manh_hips = np.sum(np.abs(lh - rh))

    tiempo_frame = time.time() - start_frame_time
    tiempo_total += tiempo_frame

    # Recursos medidos independientemente
    cpu_percent = process.cpu_percent(interval=0.0)/psutil.cpu_count()
    memory_percent = process.memory_percent()

    # Guardar en CSV
    writer.writerow([frame_num,
                     dist_euclid_shlders, dist_manh_shlders,
                     dist_euclid_lsh_hip, dist_manh_lsh_hip,
                     dist_euclid_rsh_hip, dist_manh_rsh_hip,
                     dist_euclid_hips, dist_manh_hips,
                     tiempo_frame,
                     cpu_percent, memory_percent])

    # Mostrar frame
    cv2.imshow("Video con Keypoints", frame_resized if not BW_OPTION else gray_frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_num += 1

cap.release()
cv2.destroyAllWindows()
csv_file.close()


tiempo_video_real = total_frames / fps
factor = tiempo_total / tiempo_video_real

estadisticas = f"""
Frames procesados: {frame_num}/{total_frames}
Duración del video real: {tiempo_video_real:.2f} seg
Tiempo total de procesamiento (solo inferencia y cálculo distancias): {tiempo_total:.2f} seg
Factor de procesamiento vs tiempo real: {factor:.2f}x
CSV guardado en: {OUTPUT_CSV}
"""

print(estadisticas)

# Guardar estadísticas en TXT
with open(OUTPUT_TXT, "w") as f:
    f.write(estadisticas)

print(f"📄 Estadísticas finales guardadas en: {OUTPUT_TXT}")


Video original: 848x478, FPS: 23.0, Frames: 14
Resolución ajustada a: 477x478


<>:9: SyntaxWarning: invalid escape sequence '\L'
<>:11: SyntaxWarning: invalid escape sequence '\E'
<>:12: SyntaxWarning: invalid escape sequence '\E'
<>:9: SyntaxWarning: invalid escape sequence '\L'
<>:11: SyntaxWarning: invalid escape sequence '\E'
<>:12: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17212\3638960671.py:9: SyntaxWarning: invalid escape sequence '\L'
  VIDEO_PATH = "recortes\Lado2_5m.mp4"
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17212\3638960671.py:11: SyntaxWarning: invalid escape sequence '\E'
  OUTPUT_CSV = 'Estadisticas\EstadisticasL2_5m_nano_ByN.csv'
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17212\3638960671.py:12: SyntaxWarning: invalid escape sequence '\E'
  OUTPUT_TXT = 'Estadisticas\EstadisticasL2_5m_nano_ByN.txt'



Frames procesados: 14/14
Duración del video real: 0.61 seg
Tiempo total de procesamiento (solo inferencia y cálculo distancias): 1.10 seg
Factor de procesamiento vs tiempo real: 1.81x
CSV guardado en: Estadisticas\EstadisticasL2_5m_nano_ByN.csv

📄 Estadísticas finales guardadas en: Estadisticas\EstadisticasL2_5m_nano_ByN.txt


In [12]:
import cv2
import time
import csv
from ultralytics import YOLO
import numpy as np
import os
import psutil
import glob

# ---------------------------
# CONFIGURACIÓN
# ---------------------------
VIDEO_FOLDER = "recortes"  # Carpeta con los videos
BW_OPTION = True  # True = Blanco y negro, False = Color
MODEL_PATH = "yolov8n-pose.pt"

# ---------------------------
# CARGAR MODELO
# ---------------------------
model = YOLO(MODEL_PATH)
process = psutil.Process(os.getpid())

# ---------------------------
# LISTAR VIDEOS
# ---------------------------
video_paths = glob.glob(os.path.join(VIDEO_FOLDER, "*.mp4"))

if not video_paths:
    print("❌ No se encontraron videos en la carpeta.")
    exit()

# ---------------------------
# FUNCION PARA GENERAR NOMBRES DE SALIDA
# ---------------------------
def generar_nombres(video_name, bw):
    # Extrae la parte principal del nombre
    base = os.path.splitext(os.path.basename(video_name))[0]
    if "Lado" in base:
        side = base.replace("Lado", "L").replace("_", "")
    elif "Frente" in base:
        side = "F_" + base.split("_")[1]
    elif "Espalda" in base:
        side = "E_" + base.split("_")[1]
    else:
        side = base

    color_bw = "ByN" if bw else "Color"
    csv_name = f"Estadisticas\Estadisticas{side}_nano_{color_bw}.csv"
    txt_name = f"Estadisticas\Estadisticas{side}_nano_{color_bw}.txt"
    return csv_name, txt_name

# ---------------------------
# PROCESAR CADA VIDEO
# ---------------------------
for VIDEO_PATH in video_paths:
    print(f"\nProcesando: {VIDEO_PATH}")
    OUTPUT_CSV, OUTPUT_TXT = generar_nombres(VIDEO_PATH, BW_OPTION)

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print(f"❌ No se pudo abrir el video {VIDEO_PATH}")
        continue

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    min_size = min(width, height)
    scale_w = min_size / width
    scale_h = min_size / height

    csv_headers = ["frame",
                   "dist_euclid_shlders", "dist_manh_shlders",
                   "dist_euclid_left_shoulder_hip", "dist_manh_left_shoulder_hip",
                   "dist_euclid_right_shoulder_hip", "dist_manh_right_shoulder_hip",
                   "dist_euclid_hips", "dist_manh_hips",
                   "time_frame_sec", "cpu_percent", "memory_percent"]

    with open(OUTPUT_CSV, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(csv_headers)

        frame_num = 0
        tiempo_total = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame_resized = cv2.resize(frame, (int(width*scale_w), int(height*scale_h)))

            if BW_OPTION:
                gray_frame = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2GRAY)
                proc_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
            else:
                proc_frame = frame_resized

            start_frame_time = time.time()
            results = model(proc_frame, verbose=False)

            # Inicializar distancias
            dist_euclid_shlders = dist_manh_shlders = 0
            dist_euclid_lsh_hip = dist_manh_lsh_hip = 0
            dist_euclid_rsh_hip = dist_manh_rsh_hip = 0
            dist_euclid_hips = dist_manh_hips = 0

            for result in results:
                keypoints = result.keypoints.xy.cpu().numpy()
                if len(keypoints) > 0:
                    person = keypoints[0]
                    ls = person[5].astype(float)
                    rs = person[6].astype(float)
                    lh = person[11].astype(float)
                    rh = person[12].astype(float)

                    dist_euclid_shlders = np.linalg.norm(ls - rs)
                    dist_euclid_lsh_hip = np.linalg.norm(ls - lh)
                    dist_euclid_rsh_hip = np.linalg.norm(rs - rh)
                    dist_euclid_hips = np.linalg.norm(lh - rh)

                    dist_manh_shlders = np.sum(np.abs(ls - rs))
                    dist_manh_lsh_hip = np.sum(np.abs(ls - lh))
                    dist_manh_rsh_hip = np.sum(np.abs(rs - rh))
                    dist_manh_hips = np.sum(np.abs(lh - rh))

            tiempo_frame = time.time() - start_frame_time
            tiempo_total += tiempo_frame

            cpu_percent = process.cpu_percent(interval=0.0)/psutil.cpu_count()
            memory_percent = process.memory_percent()

            writer.writerow([frame_num,
                             dist_euclid_shlders, dist_manh_shlders,
                             dist_euclid_lsh_hip, dist_manh_lsh_hip,
                             dist_euclid_rsh_hip, dist_manh_rsh_hip,
                             dist_euclid_hips, dist_manh_hips,
                             tiempo_frame, cpu_percent, memory_percent])

            frame_num += 1

    cap.release()

    tiempo_video_real = total_frames / fps
    factor = tiempo_total / tiempo_video_real

    estadisticas = f"""
Frames procesados: {frame_num}/{total_frames}
Duración del video real: {tiempo_video_real:.2f} seg
Tiempo total de procesamiento: {tiempo_total:.2f} seg
Factor de procesamiento vs tiempo real: {factor:.2f}x
CSV guardado en: {OUTPUT_CSV}
"""

    print(estadisticas)
    with open(OUTPUT_TXT, "w") as f:
        f.write(estadisticas)
    print(f"📄 Estadísticas finales guardadas en: {OUTPUT_TXT}")


<>:48: SyntaxWarning: invalid escape sequence '\E'
<>:49: SyntaxWarning: invalid escape sequence '\E'
<>:48: SyntaxWarning: invalid escape sequence '\E'
<>:49: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17212\1094902048.py:48: SyntaxWarning: invalid escape sequence '\E'
  csv_name = f"Estadisticas\Estadisticas{side}_nano_{color_bw}.csv"
C:\Users\Alejandro\AppData\Local\Temp\ipykernel_17212\1094902048.py:49: SyntaxWarning: invalid escape sequence '\E'
  txt_name = f"Estadisticas\Estadisticas{side}_nano_{color_bw}.txt"



Procesando: recortes\acercamiento_final.mp4

Frames procesados: 43/43
Duración del video real: 1.87 seg
Tiempo total de procesamiento: 3.21 seg
Factor de procesamiento vs tiempo real: 1.72x
CSV guardado en: Estadisticas\Estadisticasacercamiento_final_nano_ByN.csv

📄 Estadísticas finales guardadas en: Estadisticas\Estadisticasacercamiento_final_nano_ByN.txt

Procesando: recortes\Acomodo_a4m.mp4

Frames procesados: 41/41
Duración del video real: 1.78 seg
Tiempo total de procesamiento: 2.78 seg
Factor de procesamiento vs tiempo real: 1.56x
CSV guardado en: Estadisticas\EstadisticasAcomodo_a4m_nano_ByN.csv

📄 Estadísticas finales guardadas en: Estadisticas\EstadisticasAcomodo_a4m_nano_ByN.txt

Procesando: recortes\Dezplazamiento_a2m.mp4

Frames procesados: 107/107
Duración del video real: 4.65 seg
Tiempo total de procesamiento: 7.35 seg
Factor de procesamiento vs tiempo real: 1.58x
CSV guardado en: Estadisticas\EstadisticasDezplazamiento_a2m_nano_ByN.csv

📄 Estadísticas finales guardadas 